### Praca domowa nr 6 ###
Autor: Olga Kachniewska

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Animation_Movies.csv.zip")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51945 entries, 0 to 51944
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    51945 non-null  int64  
 1   title                 51944 non-null  object 
 2   vote_average          51945 non-null  float64
 3   vote_count            51945 non-null  int64  
 4   status                51945 non-null  object 
 5   release_date          49808 non-null  object 
 6   revenue               51945 non-null  int64  
 7   runtime               51945 non-null  int64  
 8   adult                 51945 non-null  bool   
 9   backdrop_path         15835 non-null  object 
 10  budget                51945 non-null  int64  
 11  homepage              8253 non-null   object 
 12  imdb_id               29552 non-null  object 
 13  original_language     51945 non-null  object 
 14  original_title        51944 non-null  object 
 15  overview           

In [7]:
df.describe()

,id,vote_average,vote_count,revenue,runtime,budget,popularity
count,5.194500e+04,51945.000000,51945.000000,5.194500e+04,51945.000000,5.194500e+04,51945.000000
mean,6.188122e+05,2.597083,40.395380,1.863464e+06,20.901415,5.759513e+05,1.886233
std,3.479748e+05,3.318748,482.705948,3.049824e+07,39.016002,7.606628e+06,9.700913
min,1.200000e+01,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,3.300810e+05,0.000000,0.000000,0.000000e+00,3.000000,0.000000e+00,0.600000
50%,6.119150e+05,0.000000,0.000000,0.000000e+00,7.000000,0.000000e+00,0.600000
75%,9.159040e+05,6.000000,2.000000,0.000000e+00,19.000000,0.000000e+00,0.882000
max,1.238314e+06,10.000000,19463.000000,1.450027e+09,3720.000000,2.600000e+08,1008.942000


In [6]:
df.describe(include='object')

,title,status,release_date,backdrop_path,homepage,imdb_id,original_language,original_title,overview,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
count,51944,51945,49808,15835,8253,29552,51945,51944,45866,37934,4678,51945,29398,39700,33818
unique,49381,6,16893,15731,7461,29545,102,50318,44907,37819,4451,2649,11196,1096,617
top,Little Red Riding Hood,Released,2017-01-01,/7tFOLDhZqKPactVlGfdjgVmYnlu.jpg,https://animation.geidai.ac.jp,tt27048168,en,Metamorphosis,An Aesop’s Film Fables cartoon.,/tVRb7CCGuru9RdG1ikGFCceuegy.jpg,animation short,Animation,ONF | NFB,United States of America,English
freq,15,51134,291,17,147,2,28584,9,71,12,104,28393,717,12836,12005


Z powyższych statystyk wynika, że większość filmów w zbiorze ma zerowy budżet, przychód, czas trwania, liczbę głosów i ocenę, co wskazuje na wiele braków lub niepełnych danych. Mediana długości filmu to zaledwie 7 minut, co sugeruje obecność wielu bardzo krótkich filmów (np. krótkometrażowych lub błędnie wprowadzonych danych), a skrajne wartości (np. runtime 3720 minut) mogą być odstającymi punktami wymagającymi oczyszczenia danych, natomiast  w danych typu object można zauważyć, że niektóre kolumny, ktore mają indywidualne własności filmu są powtórzenia (np. imdb_id) co może sugerować powtórzenia, a niektóre kolumny wydają się mieć zbyt dużo różnych wartości (np. production_counktries)

In [ ]:
numeric_columns = ['vote_average', 'vote_count', 'revenue', 'runtime', 'budget', 'popularity']
grouped = df.groupby('status')[numeric_columns]

group_stats = grouped.agg(['count', 'mean', 'median', 'std', 'min', 'max'])

print(group_stats)

                vote_average                                       vote_count  \
                       count      mean median       std  min   max      count   
status                                                                          
Canceled                  17  0.000000    0.0  0.000000  0.0   0.0         17   
In Production            381  0.078740    0.0  0.885018  0.0  10.0        381   
Planned                  305  0.000000    0.0  0.000000  0.0   0.0        305   
Post Production           97  0.000000    0.0  0.000000  0.0   0.0         97   
Released               51134  2.637570    0.0  3.328250  0.0  10.0      51134   
Rumored                   11  0.545455    0.0  1.809068  0.0   6.0         11   

                                               ... budget                    \
                      mean median         std  ... median           std min   
status                                         ...                            
Canceled          0.000000    0.0

Tylko filmy o statusie Released posiadają sensowne dane dotyczące ocen, budżetu, popularności i przychodów, pozostałe odzwierciedlają projekty porzucone, niedokończone lub tylko zapowiedziane — nie nadają się do analizy z punktu widzenia skuteczności filmu. Status Post Production wykazuje wyjątkowo wysoką popularność, ale brak danych o ocenach, co sugeruje filmy znane z zapowiedzi

In [15]:
# Przygotowanie kolumny genre
df['genres_clean'] = df['genres'].fillna('').str.replace('[\[\]\']', '', regex=True)
df['genres_list'] = df['genres_clean'].str.split(',\s*')

# Rozszerzenie (explode)
df_exploded = df.explode('genres_list')

# Lista zmiennych liczbowych do wyboru
numeric_columns = ['vote_count', 'revenue', 'runtime', 'budget', 'popularity']

# Funkcja rysująca wykres
def plot_genre_distribution(metric):
    data = df_exploded.groupby('genres_list')[metric].mean().sort_values(ascending=False).reset_index()
    fig = px.bar(
        data,
        x='genres_list',
        y=metric,
        title=f'Średnia wartość: {metric} według gatunku filmu',
        labels={'genres_list': 'Gatunek', metric: f'Średnia {metric}'},
        color=metric,
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

#menu
interact(plot_genre_distribution, metric=widgets.Dropdown(options=numeric_columns, description='Zmienna:'))

interactive(children=(Dropdown(description='Zmienna:', options=('vote_count', 'revenue', 'runtime', 'budget', …

<function __main__.plot_genre_distribution(metric)>

W większosci przypadków w czołówce znajdują się te same gatunki (szczególnie adventure).